In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mrfi
!pip install torch
!pip install torchvision

In [ ]:
from mrfi import MRFI, EasyConfig
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cuda:0


In [ ]:
class LeNet(nn.Module):
  def __init__(self, trained=False):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*4*4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

    if trained:
      self.load_state_dict(torch.load('/content/drive/MyDrive/LeNet/train/lenet_mnist.pth'))

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(x.size()[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
lenet = LeNet()

In [ ]:
def get_mnist(batch_size=64):
  transform = transforms.Compose(
      [transforms.ToTensor(),
       transforms.Normalize((0.1307,), (0.3081,))]
  )

  trainset = torchvision.datasets.MNIST(
      root='/content/drive/MyDrive/LeNet/data', train=True, download=True, transform=transform
  )
  testset = torchvision.datasets.MNIST(
      root='/content/drive/MyDrive/LeNet/data', train=False, download=True, transform=transform
  )

  trainloader = torch.utils.data.DataLoader(
      trainset, batch_size=batch_size, shuffle=True
  )
  testloader = torch.utils.data.DataLoader(
      testset, batch_size=batch_size, shuffle=False
  )

  return trainloader, testloader

In [ ]:
def load_dataset(dataset='mnist', batch_size=64):

  if dataset.lower()=='mnist':
    train, test = get_mnist()
    return train, test

trainloader, testloader = load_dataset()

In [ ]:
def train(epochs=1):
  lenet = LeNet()
  lenet.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(lenet.parameters(), lr=0.001)

  lenet.train()

  for epoch in range(epochs):
    running_loss = 0
    batch_size = 100
    for i, data in trainloader:
      inputs, labels = i.to(device), data.to(device)

      optimizer.zero_grad()

      outputs = lenet(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}")
  torch.save(lenet.state_dict(), '/content/drive/MyDrive/LeNet/train/lenet_mnist.pth')
train()

Epoch 1/1, Loss: 0.2310


In [ ]:
def test():
  lenet = LeNet().to(device)
  lenet.load_state_dict(torch.load('/content/drive/MyDrive/LeNet/train/lenet_mnist.pth'))
  lenet.eval()

  correct = 0
  total = 0

  with torch.no_grad():
      for images, labels in testloader:
          images, labels = images.to(device), labels.to(device)
          outputs = lenet(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          break

      print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

test()

Accuracy of the network on the 10000 test images: 100 %
